In [63]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt('Audiobooks_data.csv', delimiter=',')

unscaled_inputs_all = raw_csv_data[:, 1:-1]
targets_all = raw_csv_data[:, -1]



In [64]:
#Balance the dataset
positive_class_count = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range (targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > positive_class_count:
            indices_to_remove.append(i)

unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

In [65]:
#Standardize the inputs
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

#Shuffle the data
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]


In [66]:
#Split the dataset into train, validation, and test sets
sample_count = shuffled_inputs.shape[0]
train_samples_count = int(0.8 * sample_count)
validation_samples_count = int(0.1 * sample_count)
test_samples_count = sample_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count + validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count + validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count + validation_samples_count:]
test_targets = shuffled_targets[train_samples_count + validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print (np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count )
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1786.0 3579 0.4990220732048058
223.0 447 0.4988814317673378
228.0 448 0.5089285714285714


In [67]:
np.savez('audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('audiobooks_data_test', inputs=test_inputs, targets=test_targets)


In [68]:
import numpy as np
import tensorflow as tf


In [69]:
npz = np.load('audiobooks_data_train.npz')
train_inputs = npz['inputs'].astype(float)
train_targets = npz['targets'].astype(int)

npz = np.load('audiobooks_data_validation.npz')
validation_inputs = npz['inputs'].astype(float)
validation_targets = npz['targets'].astype(int)

npz = np.load('audiobooks_data_test.npz')
test_inputs = npz['inputs'].astype(float)
test_targets = npz['targets'].astype(int)


In [70]:
input_size = 10
output_size = 2
hidden_layer_size = 50
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
model.fit(train_inputs, train_targets, epochs=100, batch_size=100, callbacks= [early_stopping],
            validation_data=(validation_inputs, validation_targets),
            verbose=2)


Epoch 1/100
36/36 - 2s - 66ms/step - accuracy: 0.6077 - loss: 0.6434 - val_accuracy: 0.7181 - val_loss: 0.5296
Epoch 2/100
36/36 - 0s - 11ms/step - accuracy: 0.7619 - loss: 0.4794 - val_accuracy: 0.7830 - val_loss: 0.4473
Epoch 3/100
36/36 - 1s - 16ms/step - accuracy: 0.7877 - loss: 0.4228 - val_accuracy: 0.7763 - val_loss: 0.4205
Epoch 4/100
36/36 - 0s - 7ms/step - accuracy: 0.7972 - loss: 0.3983 - val_accuracy: 0.7427 - val_loss: 0.4039
Epoch 5/100
36/36 - 0s - 6ms/step - accuracy: 0.7958 - loss: 0.3823 - val_accuracy: 0.7808 - val_loss: 0.3878
Epoch 6/100
36/36 - 0s - 10ms/step - accuracy: 0.7980 - loss: 0.3699 - val_accuracy: 0.7763 - val_loss: 0.3886
Epoch 7/100
36/36 - 0s - 10ms/step - accuracy: 0.8022 - loss: 0.3645 - val_accuracy: 0.7942 - val_loss: 0.3778
Epoch 8/100
36/36 - 0s - 9ms/step - accuracy: 0.8058 - loss: 0.3594 - val_accuracy: 0.7897 - val_loss: 0.3710
Epoch 9/100
36/36 - 0s - 8ms/step - accuracy: 0.8058 - loss: 0.3503 - val_accuracy: 0.7875 - val_loss: 0.3705
Epoch

In [72]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)
print('Test loss: {0:.2f}, Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8250 - loss: 0.3403
Test loss: 0.35, Test accuracy: 81.92%
